<a href="https://colab.research.google.com/github/sonicchomp/MyImplentetion/blob/master/Convex%20Fomulations%20of%20Learning%20from%20CrowdSourcing/CFoLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.jstage.jst.go.jp/article/tjsai/27/3/27_3_133/_pdf/-char/ja


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
import theano
import theano.tensor as T
import math


In [0]:
class Optimizer(object):
    def __init__(self, params=None):
        if params is None:
            return NotImplementedError()
        self.params = params

    def updates(self, loss=None):
        if loss is None:
            return NotImplementedError()

        self.updates = OrderedDict()
        self.gparams = [T.grad(loss, param) for param in self.params]


In [0]:
class SGD(Optimizer):
    def __init__(self, learning_rate=0.01, params=None):
        super(SGD, self).__init__(params=params)
        self.learning_rate = 0.01

    def updates(self, loss=None):
        super(SGD, self).updates(loss=loss)

        for param, gparam in zip(self.params, self.gparams):
            self.updates[param] = param - self.learning_rate * gparam

        return self.updates

In [0]:
def make_W(w_0, lambd, j):
    W = np.empty((j,w_0.shape[0]))

    for i in range(j):
        w_j = np.random.normal(loc=w_0,scale=1.0,size=w_0.shape[0])
        W[i] = w_j
    return W

In [0]:
def remake_W(W,N):
  j,D = W.shape
  W_ = np.zeros((N*j,D))
  for i in range(j):
    tmp = np.tile(W[i],(N,1))
    W_[i*N: (i+1)*N] = tmp

  return W_

In [0]:
def remake_X(X,j):
  N,D = X.shape
  X_ = np.zeros((N*j,D))
  for i in range(N*j):
    X_[i,:] = X.iloc[i%N]
  return X_


In [0]:
def remake_Y(Y):
  Y_ = Y.T.values.flatten()
  return Y_

In [0]:
def model(X,y,lambd,eta,w_0,W):
  X = T.matrix(name="X")
  y = T.vector(name="y")
  w_0 = theano.shared(w_0,name="w_0")
  W = theano.shared(W,name="W")
  
  first = lambd * ((W-w_0)**2).sum()/2
  second = eta * (w_0 ** 2).sum()/2
  
  p_1 = T.nnet.nnet.sigmoid((W*X).sum(axis = 1))
  xent = T.nnet.nnet.binary_crossentropy(p_1,y)
  third = xent.mean()

  loss = first + second + third
  params = [w_0,W]
  updates = SGD(params=params).updates(loss)

  print('start: compile model')
  train = theano.function(
      inputs = [X,y],
      outputs = [loss,w_0,W],
      updates = updates,
      on_unused_input='ignore'
  )
  print('end: compile model')

  return train

In [0]:
def predict(X,y,w_0):
  logit = np.dot(X, w_0)
  y_pred = (T.nnet.nnet.sigmoid(logit)).eval()
  print(roc_auc_score(y, y_pred))



In [0]:
def numpy_model(X,y,lambd,eta,w_0,W):
  first = lambd * ((W-w_0)**2).sum()/2
  second = eta * (w_0 ** 2).sum()/2
  
  p_1 = (T.nnet.nnet.sigmoid((W*X).sum(axis = 1))).eval()
  
  xent = y * np.log(p_1) + (1-y)*np.log(1-p_1)
  xxent = (T.nnet.nnet.binary_crossentropy(p_1,y)).eval()
                                
  third = xent.sum() 

  loss = first + second - third

  tmp = np.vstack([y,p_1,xent,xxent,(W*X).sum(axis = 1),np.log(p_1)]).T
  df = pd.DataFrame(tmp,columns=["y","p_1","xent","xxent","WtX","log(p_1)"])

  import pandas_profiling as pdp
  display(pdp.ProfileReport(df))
  return
  # print("y")
  # df = pd.DataFrame(y)
  # display(df.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9]))
  # print("p_1")
  # df = pd.DataFrame(p_1)
  # display(df.describe())
  # print("xent")
  # df = pd.DataFrame(xent)
  # display(df.head())
  # print(loss)

In [0]:
def main():
    read_file = '/content/drive/My Drive/論文実装/Convex Fomulations of Learning from CrowdSourcing/winequality-red.csv'
    data = pd.read_csv(read_file, sep=';')
    Y_path = "/content/drive/My Drive/論文実装/Convex Fomulations of Learning from CrowdSourcing/Y.txt"
    Y = pd.read_csv(Y_path,sep = " ",header=None)
    X = data.drop(data.columns[-1],axis=1)
    ms = MinMaxScaler()
    X = ms.fit_transform(X)
    X = pd.DataFrame(X)
    N,D = X.shape
    # ワーカの数
    j = Y.shape[1]
    y_index = range(N)
    X_train, X_test, y_train_index, y_test_index = train_test_split(X,y_index,shuffle = True,random_state = 1)
    Y_train = Y.iloc[y_train_index,:]
    Y_test = Y.iloc[y_test_index]
    
    eta = 0.01
    lambd = 0.01
    training_epochs = 100
    np.random.seed(seed=38)
    w_0 = np.random.normal(loc=0.0,scale=eta,size=X_train.shape[1])
    W = make_W(w_0,lambd,j)
    W_ = remake_W(W,X_train.shape[0])
    X_train = remake_X(X_train,j)
    Y_train = remake_Y(Y_train)
    
    train = model(X_train,Y_train,lambd,eta,w_0,W_)
    # numpy_model(X_train,Y_train,lambd,eta,w_0,W_)
    min_w = np.empty_like(w_0)
    min_loss = 999
    print("epochs: {}".format(training_epochs))
    
    for t in range(training_epochs):
        loss, w_0, W = train(X_train,Y_train)
        if t % (training_epochs/10) == 0:
            print('{}: loss:{}'.format(t,loss))
        if loss < min_loss :
            min_w = w_0
            min_loss = loss

    
    # print("-"*20)
    # test = test_model(X_test,y_test,lambd,min_w)
    # test_loss, prediction = test(X_test,y_test)
    # print("test loss: {}".format(test_loss))      
    # cm = confusion_matrix(prediction, y_test) 
    # print("Accuracy: {}".format(cm.accuracy()))

    print('-'*20)
    X_test = remake_X(X_test,j)
    Y_test = remake_Y(Y_test)
    predict(X_test,Y_test,min_w)
    pass

Pr[w_0|\eta] = N(0,\eta^-1 I]  
Pr[w_j|w_o,lambd] =  N(w_0,\lambd I]  
Pr[y|x_i, w_j]

In [39]:
main()

start: compile model
end: compile model
epochs: 100
0: loss:917.8194046542839
10: loss:821.792956906894
20: loss:820.1522501129872
30: loss:818.5148221450722
40: loss:816.8806664513551
50: loss:815.249776493229
60: loss:813.6221457450897
70: loss:811.9977676943089
80: loss:810.3766358414448
90: loss:808.7587436998984
--------------------
0.7198065958570161
